<a href="https://colab.research.google.com/github/diwert-ai/First-step-in-NLP-HSE-bootcamp-/blob/main/kaggle%20competition/experiment_VotingClassifier%5BLB_0_7730%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mount google.drive and copy dataset to colab linux instance

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
INPUT_DIR ='/content/drive/MyDrive/kaggle/fsnlp_hse/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/fsnlp_hse/output/'

WORK_DIR='/content/'
EXPERIMENT='votingcl'
EXPERIMENT_DIR = OUTPUT_DIR + EXPERIMENT+'/'

DATASET_DIR = INPUT_DIR
METADATA_DIR = INPUT_DIR

In [ ]:
!mkdir {EXPERIMENT_DIR}

# Install and set up Kaggle API

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!pip install -q kaggle --upgrade
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [ ]:
!kaggle competitions download -c hse-nlp-bootcamp
!unzip hse-nlp-bootcamp.zip

 97% 58.0M/59.8M [00:00<00:00, 202MB/s]
100% 59.8M/59.8M [00:00<00:00, 183MB/s]
Archive:  hse-nlp-bootcamp.zip
  inflating: new_test_ml.csv         
  inflating: sample_submission.csv   
  inflating: train_ml.csv            


# Imports

In [ ]:
import pandas as pd
import numpy as np
import random
import pprint as pp
import os


# import re
# from pymorphy2 import MorphAnalyzer
# from functools import lru_cache
# from nltk.corpus import stopwords

# from multiprocessing import Pool
# from tqdm import tqdm

import nltk
# nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, f1_score
from joblib import dump, load

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Configuration

In [ ]:
class Config:
  seed = 12536271
  samples_num = 51476
  experiment = EXPERIMENT
  test_size = 0.1

# Main pipeline starts here

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(Config.seed)

## Load and process train data

In [ ]:
df = pd.read_csv('/content/train_ml.csv')
banks_list = list(df.bank.unique())
df['bank_id'] = df['bank'].apply(lambda x: banks_list.index(x))
df['date'] = pd.to_datetime(df['date'], format='%d.%m.%Y %H:%M')
df = df.dropna()
df = df.astype({'grades': 'int32'})
df.head()

,bank,feeds,grades,date,bank_id
0,ubrr,"Много лет являюсь клиентом этого банка, но пос...",1,2017-02-16 16:10:00,0
1,fk_otkritie,"Г. Ростов-на-Дону, ул. Ленина, 48. Были 10.12....",2,2016-12-13 01:05:00,1
4,promsvyazbank,"Имею потребительский кредит, взятый в Связь-ба...",2,2020-08-04 06:38:00,4
7,homecreditbank,Хочу выразить глубокую благодарность всем сотр...,5,2015-04-19 20:11:00,7
8,vtb,"6 марта 2019г. В 10:33 при проходе в метро ""Ку...",2,2019-03-28 18:36:00,3


In [ ]:
len(df)

51476

In [ ]:
subset = df.sample(Config.samples_num, random_state=Config.seed)
subset['text'] = subset.feeds.apply(lambda x: x.lower())
subset.reset_index(drop=True, inplace=True)

## Split data on train and test data

In [ ]:
if Config.test_size > 0.0:
  x_train, x_test, y_train, y_test = train_test_split(subset[['bank_id','text']], subset['grades'], test_size=Config.test_size, random_state=Config.seed)
else:
  x_train, y_train, x_test, y_test = subset[['bank_id','text']], subset['grades'], None, None

In [ ]:
len(x_train)

46328

## Set up train pipeline

In [ ]:
def get_numeric_data(x):
    return x[['bank_id']]

def get_text_data(x):
    return x['text']

transfomer_numeric = FunctionTransformer(get_numeric_data)
transformer_text = FunctionTransformer(get_text_data)

clf1 = LogisticRegression(random_state=Config.seed,  warm_start=True, max_iter=1000)
clf2 = RandomForestClassifier(random_state=Config.seed)
clf3 = SGDClassifier(random_state=None, max_iter=1000, loss='log')

pipeline = Pipeline([
    ('features', FeatureUnion([
            ('numeric_features', Pipeline([
                ('selector', transfomer_numeric)
            ])),
             ('text_features', Pipeline([
                ('selector', transformer_text),
                ('vec', TfidfVectorizer(ngram_range=(1, 2), tokenizer=word_tokenize))
            ]))
         ])),
    ('vclf', VotingClassifier(estimators=[('lr', clf1), ('rf', clf2),('sgd', clf3)], voting='soft'))
])

In [ ]:
pp.pprint(sorted(pipeline.get_params().keys()))

In [ ]:
param_grid = {'features__text_features__vec__max_df': [1.0],
              'features__text_features__vec__min_df': [0.001],
              'vclf__lr__penalty': ['l2'],
              'vclf__lr__C': [10.0],
              'vclf__rf__criterion': ['gini'],
              'vclf__rf__n_estimators': [100], 
              'vclf__sgd__loss' : ['modified_huber'],
              'vclf__sgd__penalty': ['l1'],
              }

kfold = StratifiedKFold(n_splits=5)
scoring = {'F1': 'f1_micro'}
refit = 'F1'

## Perform GridSearch

In [ ]:
print(f'samples: {Config.samples_num}')
lr_model = GridSearchCV(pipeline, param_grid=param_grid, cv=kfold, scoring=scoring, 
                         refit=refit, n_jobs=-1, return_train_score=True, verbose=10)
lr_model.fit(x_train, y_train)
lr_best = lr_model.best_estimator_

samples: 51476
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## Test best model

In [ ]:
if Config.test_size > 0.0:
  pred = lr_best.predict(x_test)
  print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           1       0.77      0.94      0.85      2788
           2       0.31      0.08      0.13       582
           3       0.26      0.08      0.12       253
           4       0.18      0.12      0.14       135
           5       0.88      0.92      0.90      1390

    accuracy                           0.77      5148
   macro avg       0.48      0.43      0.43      5148
weighted avg       0.71      0.77      0.73      5148



In [ ]:
if Config.test_size > 0.0:
  print(f1_score(y_test, pred, average='micro'))

0.7736985236985238


In [ ]:
lr_model.best_params_

{'features__text_features__vec__max_df': 1.0,
 'features__text_features__vec__min_df': 0.001,
 'vclf__lr__C': 10.0,
 'vclf__lr__penalty': 'l2',
 'vclf__rf__criterion': 'gini',
 'vclf__rf__n_estimators': 100,
 'vclf__sgd__loss': 'modified_huber',
 'vclf__sgd__penalty': 'l1'}

In [ ]:
lr_model.cv_results_

{'mean_fit_time': array([558.69423919]),
 'std_fit_time': array([63.46163086]),
 'mean_score_time': array([23.26749315]),
 'std_score_time': array([3.69382426]),
 'param_features__text_features__vec__max_df': masked_array(data=[1.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_features__text_features__vec__min_df': masked_array(data=[0.001],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_vclf__lr__C': masked_array(data=[10.0],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_vclf__lr__penalty': masked_array(data=['l2'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_vclf__rf__criterion': masked_array(data=['gini'],
              mask=[False],
        fill_value='?',
             dtype=object),
 'param_vclf__rf__n_estimators': masked_array(data=[100],
              mask=[False],
        fill_value='?',
             dtype=obj

In [ ]:
lr_model.best_score_

0.7729881773322125

In [ ]:
cv_tag = f'[cv_{lr_model.best_score_:0.4f}]'

In [ ]:
cv_tag

'[cv_0.7730]'

## Inference (calculating submission) 

In [ ]:
test = pd.read_csv('/content/new_test_ml.csv', index_col=0)
test['text'] = test.feeds.apply(lambda x: x.lower())
test['bank_id'] = test['bank'].apply(lambda x: banks_list.index(x))
test = test[['bank_id', 'text']]
test_pred = lr_best.predict(test)
probs = lr_best.predict_proba(test)
sub = pd.DataFrame({'inds': test.index,
                    'grades': test_pred})
sub

,inds,grades
0,0,5
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
17215,17215,1
17216,17216,1
17217,17217,1
17218,17218,1


# Save results to files

In [ ]:
suffix = f'{Config.experiment}_[sn_{Config.samples_num}]_[sd_{Config.seed}]'+cv_tag
sub_filename = 'sub_'+suffix+'.csv'
probs_filename = 'probs_'+suffix+'.npy'
model_filename = 'model_'+suffix+'.joblib'

In [ ]:
sub.to_csv(sub_filename, index=False)
np.save(probs_filename, probs)
dump(lr_best, model_filename) 

['model_votingcl_[sn_51476]_[sd_12536271][cv_0.7730].joblib']

In [ ]:
!cp {sub_filename} {EXPERIMENT_DIR}
!cp {probs_filename} {EXPERIMENT_DIR}
!cp {model_filename} {EXPERIMENT_DIR}

# Submit predictions

In [ ]:
!kaggle competitions submit -c hse-nlp-bootcamp  -f {'/content/'+sub_filename} -m "Submission via API"

100% 124k/124k [00:00<00:00, 232kB/s]
Successfully submitted to HSE | NLP Bootcamp | Sentiment Analysis

# Submit CV ~ 0.79 - best public LB 0.77862